# MLOps in BQML - e-commerce scenario

<table align="left">
  <td>
    <a>
       <img src="https://vitalflux.com/wp-content/uploads/2021/09/google-cloud-bigquery-ml.png" style="max-width: 75%; height: auto;">
    </a>
  </td> 
</table>

## Overview

The purpose of this lab is to go through capabilities of BigQuery ML when modelling and maintaining a machine learning model based on tabular data. Throughout this lab, you will learn how to read historical e-commerce data stored in data warehouse, perform exploratory data analysis (EDA), do feature engineering, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do inference on your model with feature store, and monitor your model.

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)
- [Google Cloud Storage](https://cloud.google.com/storage)

### Import packages

Import the following packages required to execute this notebook.

In [ ]:
#pip install --upgrade google-api-core

In [ ]:
#pip install --upgrade pandas-gbq

In [ ]:
import pandas as pd
import pandas_gbq
import random
import numpy as np
import google.cloud.aiplatform as aip

from pandas import read_gbq
from google.cloud import bigquery
from typing import Union
from google.cloud import aiplatform as vertex_ai
from google.cloud.aiplatform import gapic as aip_gapic
from google.cloud.aiplatform import model_monitoring

### Setup your environment

Run the next cell to set your project ID and some of the other constants used in the lab.

#### >>> PLEASE MODIFY FOLLOWING VARIABLES <<<

In [ ]:
# THESE
PROJECT_ID = "dprzek-vertex"
DATASET_NAME = "workshop_ecommerce"
TABLE_NAME = "trans_sample"
MODEL_NAME = "ecommerce_workshop_model"
ENDPOINT_NAME = "endpoint_ecommerce_workshop"
MONITORING_JOB_NAME = "ecommerce_monit"

In [ ]:
# NOT THESE
TARGET = "purchase"

REGION = "us-central1"
BUCKET_NAME = 'ecommerce_workshop_bucket'

# create a Cloud Storage bucket
!gsutil mb -l {REGION} gs://{BUCKET_NAME}

#### Create helper function

In [ ]:
# wrapper to use BigQuery client to run query/job, return job ID or result as DF
def run_bq_query(sql: str, show=False) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
        show: A flag to show query result in a Pandas Dataframe
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    result = client_result.result()
    print(f"Finished job_id: {job_id}")
    
    if show:
        df = result.to_arrow().to_pandas()
        return df

##### Transactions data table

In [ ]:
run_bq_query(
    f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}` as
    SELECT
      sessions.fullVisitorId,
      sessions.ga_session_id,
      sessions.churned,
      sessions.event_date,
      sessions.isMobile,
      sessions.operatingSystem,
      sessions.browser,
      sessions.country,
      sessions.city,
      sessions.firstSource,
      sessions.firstMedium,
      sessions.sessionNumber,
      sessions.latest_ecommerce_progress,
      sessions.isFirstVisit,
      sessions.productPagesViewed,
      sessions.addedToCart,
      sessions.purchase,

      users.totalHits,
      users.totalPageviews,

      visits.totalVisits,

      engagement.totalTimeOnSite, # bucket
      engagement.engagement_seconds,
      engagement.engagement_minutes,
      engagement.engagement_avg_seconds,
      engagement.engagement_avg_minutes,

      source.source,
      source.medium,
      source.campaign,

    FROM (
          SELECT
            fullVisitorId,
            session_id,
            ga_session_id,
            churned,
            event_date,
            isMobile,
            operatingSystem,
            browser,
            country,
            city,
            firstSource,
            firstMedium,
            MAX(ga_session_number) sessionNumber,
            MAX(latest_ecommerce_progress) latest_ecommerce_progress,
            MAX(isFirstVisit) isFirstVisit,
            MAX(productPagesViewed) productPagesViewed,
            MAX(addedToCart) addedToCart,
            MAX(purchase) purchase
          FROM (
                SELECT
                  user_pseudo_id fullVisitorId,
                  CONCAT(user_pseudo_id, "-", (SELECT value.int_value FROM unnest(event_params) WHERE key="ga_session_id")) session_id,
                  (select value.int_value from unnest(event_params) where key = 'ga_session_id') ga_session_id,
                  (select value.int_value from unnest(event_params) where key = 'ga_session_number') ga_session_number,
                  IF (TIMESTAMP_MICROS(event_timestamp) < TIMESTAMP_ADD(TIMESTAMP_MICROS(user_first_touch_timestamp), INTERVAL 24 HOUR), 1, 0 ) churned,
                  event_date,
                  CASE device.category WHEN 'mobile' THEN 0  ELSE 1 END AS isMobile,
                  device.operating_system operatingSystem,
                  device.web_info.browser browser,
                  geo.country AS country,
                  IFNULL(geo.city, '') city,
                  traffic_source.source firstSource,
                  traffic_source.medium firstMedium,
                  CASE event_name WHEN 'first_visit' THEN 1  ELSE 0 END isFirstVisit,
                  CASE event_name WHEN 'view_item' THEN 1 ELSE 0 END productPagesViewed,
                  CASE event_name WHEN 'add_to_cart' THEN 1  ELSE 0 END addedToCart,
                  CASE event_name WHEN 'purchase' THEN 1  ELSE 0 END purchase,
                  CASE
                    WHEN event_name = 'view_item' THEN 1
                    WHEN event_name = 'add_to_cart' THEN 2
                    WHEN event_name = 'view_cart' THEN 3
                    WHEN event_name = 'begin_checkout' THEN 4
                    WHEN event_name = 'purchase' THEN 5
                  ELSE 0 END AS latest_ecommerce_progress

              FROM
                `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
              WHERE
                platform = "WEB"
                and (select value.int_value from unnest(event_params) where key = 'ga_session_id') IS NOT NULL
                -- and user_pseudo_id = '335679038.1695746809'
              GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18)

              GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12
            ) sessions

    LEFT JOIN (

        SELECT
        user_pseudo_id fullVisitorId,
        CONCAT(user_pseudo_id, "-", (SELECT value.int_value FROM unnest(event_params) WHERE key="ga_session_id")) session_id,
        COUNT(user_pseudo_id) totalHits,
        COUNTIF(event_name = 'page_view') totalPageviews
        FROM
          `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
        WHERE
          platform = "WEB"
          -- and user_pseudo_id = '335679038.1695746809'
        GROUP BY 1,2
    ) users USING(session_id)

    LEFT JOIN (

        SELECT
        user_pseudo_id AS fullVisitorId,
        CONCAT(user_pseudo_id, "-", (SELECT value.int_value FROM unnest(event_params) WHERE key="ga_session_id")) session_id,
        COUNT(DISTINCT (select value.int_value from unnest(event_params) where key = 'ga_session_id')) totalVisits
        FROM
          `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
        WHERE
        platform = "WEB"
        -- and user_pseudo_id = '335679038.1695746809'
        GROUP BY 1,2
    ) visits USING(session_id)

    LEFT JOIN (

        SELECT
        user_pseudo_id fullVisitorId,
        session_id,
        MAX(engagement_time_msec) totalTimeOnSite,
        cast(coalesce(sum(engagement_time_seconds),0) as INT64) engagement_seconds,
        cast(coalesce(sum(engagement_time_minutes),0) as INT64) engagement_minutes,
        cast(coalesce(avg(engagement_time_seconds),0) as INT64) engagement_avg_seconds,
        cast(coalesce(avg(engagement_time_minutes),0) as INT64) engagement_avg_minutes
        FROM (
              SELECT
              user_pseudo_id,
              CONCAT(user_pseudo_id, "-", (SELECT value.int_value FROM unnest(event_params) WHERE key="ga_session_id")) session_id,
              (select value.int_value from unnest(event_params) where key = 'engagement_time_msec') engagement_time_msec,
              max((select value.string_value from unnest(event_params) where key = 'session_engaged')) as session_engaged,
              sum((select value.int_value from unnest(event_params) where key = 'engagement_time_msec'))/1000 as engagement_time_seconds,
              sum((select value.int_value from unnest(event_params) where key = 'engagement_time_msec'))/(1000 * 60) as engagement_time_minutes
              FROM
              `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
              WHERE
              platform = "WEB"
              -- and user_pseudo_id = '335679038.1695746809'
              group by 1,2,3
        )
        GROUP BY 1,2
    ) engagement USING(session_id)

    LEFT JOIN (

        SELECT
        user_pseudo_id AS fullVisitorId,
        CONCAT(user_pseudo_id, "-", (SELECT value.int_value FROM unnest(event_params) WHERE key="ga_session_id")) session_id,
        (array_agg((select value.string_value from unnest(event_params) where key = 'source') ignore nulls order by event_timestamp)[offset(0)]) as source,
        (array_agg((select value.string_value from unnest(event_params) where key = 'medium') ignore nulls order by event_timestamp)[offset(0)]) as medium ,
        (array_agg((select value.string_value from unnest(event_params) where key = 'campaign') ignore nulls order by event_timestamp)[offset(0)]) as campaign
        FROM
          `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`
        WHERE
        platform = "WEB"
        -- and user_pseudo_id = '335679038.1695746809'
        GROUP BY 1,2
    ) source USING(session_id)
    """, show=True
)

### Exploratory data analysis of transaction data in BigQuery

In this section, you'll explore the data by running queries and creating a couple of plots.

#### Let’s add some calculated metrics:

1. Average Price = Total Revenue / Total Quantity Sold
2. Average Quantity Per Order = Average Quantity Per Order = Total Quantity Sold / Number of Orders
3. Cart Conversion Rate = (Number of Orders / Add to Carts) * 100
4. Cart Abandonment Rate = (Add to Carts — Number of Orders) / Add to Carts) * 100

The following query will add columns and do the calculations for the desired output.

In [ ]:
run_bq_query(
    """
    DECLARE date1, date2 STRING;
    SET (date1, date2) = ('20000101','20240630');

    -- CTE for sales data excluding items with "(not set)" in item_id or item_name
    WITH sales AS (
      SELECT
        items.item_id AS item_id,
        SUM(items.quantity) AS total_quantity,
        COUNT(DISTINCT event_timestamp) AS orders,
        SUM(items.item_revenue) AS total_revenue
      FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`,
      UNNEST(items) AS items
      WHERE _TABLE_SUFFIX BETWEEN date1 AND date2
        AND event_name = 'purchase'
        AND items.item_id IS NOT NULL
        AND items.item_name IS NOT NULL
        AND items.item_id != '(not set)'
        AND items.item_name != '(not set)'
      GROUP BY item_id
    ),

    -- CTE for add-to-carts data excluding items with "(not set)" in item_id or item_name
    add_to_carts AS (
      SELECT
        items.item_id AS item_id,
        items.item_name AS item_name,
        COUNT(DISTINCT event_timestamp) AS add_to_carts
      FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`,
      UNNEST(items) AS items
      WHERE _TABLE_SUFFIX BETWEEN date1 AND date2
        AND event_name = 'add_to_cart'
        AND items.item_id IS NOT NULL
        AND items.item_name IS NOT NULL
        AND items.item_id != '(not set)'
        AND items.item_name != '(not set)'
      GROUP BY item_id, item_name
    )

    -- Final SELECT to combine sales and add-to-carts data
    SELECT
      c.item_id AS item_id,
      c.item_name AS item_name,
      c.add_to_carts AS add_to_carts,
      IFNULL(s.orders, 0) AS orders,
      IFNULL(s.total_quantity, 0) AS total_quantity_ordered,
      IFNULL(s.total_revenue, 0.0) AS revenue,
      -- Calculate average price (revenue divided by quantity)
      IFNULL(s.total_revenue / NULLIF(s.total_quantity, 0), 0.0) AS avg_price,
      -- Calculate average quantity per order
      IFNULL(s.total_quantity / NULLIF(s.orders, 0), 0.0) AS avg_quantity_per_order,
      -- Calculate cart conversion rate (orders divided by add to carts)
      IFNULL(s.orders / NULLIF(c.add_to_carts, 0), 0.0) AS cart_conversion_rate,
      -- Calculate cart abandonment rate ensuring it is not negative
      GREATEST(1 - IFNULL(s.orders / NULLIF(c.add_to_carts, 0), 0.0), 0) AS cart_abandonment_rate
    FROM add_to_carts AS c
    LEFT JOIN sales AS s ON c.item_id = s.item_id
    ORDER BY orders DESC;
    """, show=True
)

#### < space for further EDA >

### Creating logistic regression model in BigQuery

The procedure can (or even should!) be augmented with feature engineering steps.

In [ ]:
run_bq_query(
    f"""
    CREATE OR REPLACE MODEL
      `{PROJECT_ID}.{DATASET_NAME}.{MODEL_NAME}`

    OPTIONS(MODEL_TYPE = 'LOGISTIC_REG',
        INPUT_LABEL_COLS = ['purchase'],
        enable_global_explain=TRUE,
        model_registry='vertex_ai'
        )
        AS

    SELECT
      *
    FROM
      `{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}`
    WHERE 
      MOD(ABS(FARM_FINGERPRINT(CAST(event_date AS STRING))),1000) > 900
    """, show=True
)

##### A brief look at feature importance

In [ ]:
run_bq_query(
    f"""
    SELECT
      *
    FROM
      ML.GLOBAL_EXPLAIN(MODEL `{PROJECT_ID}.{DATASET_NAME}.{MODEL_NAME}`)
    """, show=True
)

#### Export model to Cloud Storage

In [ ]:
run_bq_query(
    f"""
    EXPORT MODEL `{PROJECT_ID}.{DATASET_NAME}.{MODEL_NAME}` 
    OPTIONS(URI = 'gs://{BUCKET_NAME}/model')
    """, show=True
)

#### Register model in Vertex AI Model Registry

In [ ]:
# Create Model Resource
model = aip.Model.upload(
    display_name=MODEL_NAME,
    artifact_uri= f'gs://{BUCKET_NAME}/model',
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-8:latest",  
)

#### Deploy model to Vertex AI endpoint

In [ ]:
#model = vertex_ai.Model(model_name=model.resource_name)

endpoint = model.deploy(
    deployed_model_display_name=ENDPOINT_NAME,
    machine_type="n1-standard-4",  # Or choose a suitable machine type
    min_replica_count=1,
    max_replica_count=1,
)

### Deploy model monitoring

##### Put BigQuery table into Pandas data frame

In [ ]:
query = f"""
    SELECT * 
    FROM `{DATASET_NAME}.{TABLE_NAME}`
"""

# Read from BigQuery into Pandas DataFrame
df = pd.read_gbq(query=query,project_id=PROJECT_ID)

#print(df.head())

##### Take a brief look at data frame metadata

In [ ]:
df.info()

##### Convert all object columns to string

In [ ]:
df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).astype('string')
df.info()

##### Drop target column

In [ ]:
df_nt = df.drop(TARGET, axis=1)

##### Transform testing examples into list od dictionaries

In [ ]:
# Extract specific rows
start_row = 17000
end_row = 18000
extracted_df = df_nt.iloc[start_row:end_row].copy()

# Get column names and their data types
column_dtypes = extracted_df.dtypes.to_dict()

# Fill missing values in the extracted DataFrame (if any), handling StringArrays
for col_name, col_dtype in column_dtypes.items():
    if col_dtype == 'string':  # or isinstance(extracted_df[col_name].dtype, pd.StringDtype) 
        extracted_df[col_name] = extracted_df[col_name].fillna('')  # Fill with empty string
    else:
        extracted_df[col_name] = extracted_df[col_name].fillna(0)  # Fill numeric columns with 0

# Get column names and their data types
column_dtypes = extracted_df.dtypes.to_dict()

# Create the list of dictionaries
abc = []
for _, row in extracted_df.iterrows():
    row_dict = {}
    for col_name, col_value in row.items():
        # Convert data types based on column_dtypes
        if column_dtypes[col_name] == 'Int64':
            row_dict[col_name] = int(col_value) if pd.notnull(col_value) else 0 
        elif column_dtypes[col_name] == 'Float64':
            row_dict[col_name] = float(col_value) if pd.notnull(col_value) else 0.0 
        else:
            row_dict[col_name] = str(col_value) if pd.notnull(col_value) else ""

    abc.append(row_dict)
    
#endpoint.predict(instances=abc)

## Monitoring

##### Define prerequisities to monitoring job

In [ ]:
# Sampling rate (optional, default=.8)
LOG_SAMPLE_RATE = 0.8  # @param {type:"number"}

# Monitoring Interval in hours (optional, default=1).
MONITOR_INTERVAL = 1  # @param {type:"number"}

# Skew and drift thresholds.

SKEW_THRESHOLDS = {
    "churned": 0.003,
    "isMobile": 0.003,
    "sessionNumber": 0.003,
    "latest_ecommerce_progress": 0.003, 
    "isFirstVisit": 0.003,
    "productPagesViewed": 0.003,
    "addedToCart": 0.003,
    "totalHits": 0.003,
    "totalPageviews": 0.003,
    "totalVisits": 0.003,
    "totalTimeOnSite": 0.003,
    "engagement_seconds": 0.003,
    "engagement_minutes": 0.003,
    "engagement_avg_seconds": 0.003,
    "engagement_avg_minutes": 0.003,
}
DRIFT_THRESHOLDS = {
    "churned": 0.003,
    "isMobile": 0.003,
    "sessionNumber": 0.003,
    "latest_ecommerce_progress": 0.003, 
    "isFirstVisit": 0.003,
    "productPagesViewed": 0.003,
    "addedToCart": 0.003,
    "totalHits": 0.003,
    "totalPageviews": 0.003,
    "totalVisits": 0.003,
    "totalTimeOnSite": 0.003,
    "engagement_seconds": 0.003,
    "engagement_minutes": 0.003,
    "engagement_avg_seconds": 0.003,
    "engagement_avg_minutes": 0.003,
}
ATTRIB_SKEW_THRESHOLDS = {
    "churned": 0.003,
    "isMobile": 0.003,
    "sessionNumber": 0.003,
    "latest_ecommerce_progress": 0.003, 
    "isFirstVisit": 0.003,
    "productPagesViewed": 0.003,
    "addedToCart": 0.003,
    "totalHits": 0.003,
    "totalPageviews": 0.003,
    "totalVisits": 0.003,
    "totalTimeOnSite": 0.003,
    "engagement_seconds": 0.003,
    "engagement_minutes": 0.003,
    "engagement_avg_seconds": 0.003,
    "engagement_avg_minutes": 0.003,
}
ATTRIB_DRIFT_THRESHOLDS = {
    "churned": 0.003,
    "isMobile": 0.003,
    "sessionNumber": 0.003,
    "latest_ecommerce_progress": 0.003, 
    "isFirstVisit": 0.003,
    "productPagesViewed": 0.003,
    "addedToCart": 0.003,
    "totalHits": 0.003,
    "totalPageviews": 0.003,
    "totalVisits": 0.003,
    "totalTimeOnSite": 0.003,
    "engagement_seconds": 0.003,
    "engagement_minutes": 0.003,
    "engagement_avg_seconds": 0.003,
    "engagement_avg_minutes": 0.003,
}

In [ ]:
skew_config = model_monitoring.SkewDetectionConfig(
    data_source=f"bq://{PROJECT_ID}.{DATASET_NAME}.{TABLE_NAME}",
    skew_thresholds=SKEW_THRESHOLDS,
    attribute_skew_thresholds=ATTRIB_SKEW_THRESHOLDS,
    target_field=TARGET,
)

drift_config = model_monitoring.DriftDetectionConfig(
    drift_thresholds=DRIFT_THRESHOLDS,
    attribute_drift_thresholds=ATTRIB_DRIFT_THRESHOLDS,
)

objective_config = model_monitoring.ObjectiveConfig(
    skew_config, drift_config
)

# Create sampling configuration
random_sampling = model_monitoring.RandomSampleConfig(sample_rate=LOG_SAMPLE_RATE)

# Create schedule configuration
schedule_config = model_monitoring.ScheduleConfig(monitor_interval=MONITOR_INTERVAL)

# Create alerting configuration.
emails = ["dprzek@google.com"]
alerting_config = model_monitoring.EmailAlertConfig(
    user_emails=emails, enable_logging=True
)

#### Create monitoring job

In [ ]:
job_new = vertex_ai.ModelDeploymentMonitoringJob.create(
    display_name=MONITORING_JOB_NAME,
    logging_sampling_strategy=random_sampling,
    schedule_config=schedule_config,
    alert_config=alerting_config,
    objective_configs=objective_config,
    project=PROJECT_ID,
    location=REGION,
    endpoint=endpoint,
)

#### Request the endpoint

In [ ]:
endpoint.predict(instances=abc)

In [ ]:
# #close resources:

# # [1] storage bucket
# !gsutil -m rm -r gs://{BUCKET_NAME}/**
# !gsutil rb gs://{BUCKET_NAME}

# # [2] models and endpoints
# def delete_all_models(PROJECT_ID, REGION):

#     vertex_ai.init(project=PROJECT_ID, location=REGION)
#     endpoints = vertex_ai.Endpoint.list()  # Get all endpoints

#     for endpoint in endpoints:
#         endpoint.undeploy_all()
#         print(f"Undeployed endpoints")
    
#     for model in vertex_ai.Model.list():
#         model.delete()
#         print(f"Deleted model: {model.name}")

# delete_all_models(PROJECT_ID, REGION)

# # [3] experiments
# experiments = vertex_ai.Experiment.list()
# for experiment in experiments:
#     experiment.delete(delete_backing_tensorboard_runs=True)

# # [4] datasests
# def delete_all_datasets():
#     try:
#         datasets = vertex_ai.TabularDataset.list() # or ImageDataset, TextDataset, etc.

#         for dataset in datasets:
#             dataset.delete()
#             print(f"Deleted dataset: {dataset.name}")

#         print("All datasets deleted successfully.")

#     except Exception as e:
#         print(f"Error deleting datasets: {e}")

# delete_all_datasets()

# # [5] featurestores
# featurestores = vertex_ai.Featurestore.list() 
# for featurestore in featurestores:
#     featurestore.delete(force=True)  # Delete with force to bypass checks

# # [6] artifact registry
# client = artifactregistry_v1.ArtifactRegistryClient()
# repositories = client.list_repositories(parent=f"projects/{PROJECT_ID}/locations/{REGION}")
# for repository in repositories:
#     try:
#         client.delete_repository(name=repository.name)
#         print(f"Deleted repository: {repository.name}")
#     except Exception as e:
#         print(f"Error deleting repository {repository.name}: {e}")
#     print(f"Deleted featurestore: {featurestore.name}")

# # [7] metadata
# # Initialize Metadata Service Client
# metadata_client = MetadataServiceClient(client_options={"api_endpoint": f"{REGION}-aiplatform.googleapis.com"})

# def delete_artifacts(metadata_store_id="default"):  # Use 'default' for the default store
#     parent = f"projects/{PROJECT_ID}/locations/{REGION}/metadataStores/{metadata_store_id}"
#     artifacts = metadata_client.list_artifacts(parent=parent)

#     for artifact in artifacts:
#         metadata_client.delete_artifact(name=artifact.name)
#         print(f"Deleted artifact: {artifact.name}")
       
# delete_artifacts()